In [1]:
from sympy import *
import scipy
import numpy as np
import math

In [2]:
init_printing(use_unicode=True, wrap_line=False)

In [3]:
x, y, z = symbols("x y z")

In [7]:
def GradDescend(f, x0:np.ndarray, lr:float = 0.1, eps:float = 1e-06) -> np.ndarray:
    #Считаем производные прямо здесь
    _dx = [diff(f,x), diff(f,y)]
    while True:
        dx = np.array([float(_dx[0].subs(x, x0[0])), float(_dx[1].subs(y, x0[1]))])
        x0 = x0 - lr * dx
        if np.linalg.norm(dx) < eps:
            break
    return x0

In [8]:
function = x**2 + y**2

In [9]:
GradDescend(function, np.array([100,500]))

array([7.77067557e-08, 3.88533778e-07])